In [1]:
%load_ext autoreload

In [6]:
# import Inicializacion
# # %matplotlib inline
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# from numpy.ma.bench import timer
# from scipy.sparse.linalg import svds
# import category_encoders as ce
# from xgboost import XGBClassifier, XGBRegressor
# from sklearn import metrics
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from sklearn.metrics import roc_auc_score
# from sklearn.model_selection import StratifiedKFold
# from datetime import datetime
from Inicializacion import *
from Prediction import *
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from ParamsGrid import *

In [10]:
# out = Inicializacion.Inicializacion()
preprocesamiento = Inicializacion()
data = preprocesamiento.getDataframes()

Inicializando dataframes
Comenzando operaciones
	 Nulls
		 Drop cols
		 Fill metros
			 Terreno
			 Apartamento
			 Metros cubiertos null
			 Metros totales null
			 Terreno comercial
	 Encoding
	 drop nans in selected columns
		 Drop nan
	 Cast
	 Features engineering
		 Separar fecha
		 Predict nulls
			 fill with xgboost. Feature:  {'garages'}
			 Time taken: 0 hours 1 minutes and 5.54 seconds.
				 Los mejores parametros son: 
				 {'subsample': 0.8, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.5, 'colsample_bytree': 0.8}
				 ------------------------
				 Best score: 
				 0.6687381902162503
				 ------------------------
			 fill with xgboost. Feature:  {'banos'}
			 Time taken: 0 hours 1 minutes and 29.3 seconds.
				 Los mejores parametros son: 
				 {'subsample': 0.8, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.5, 'colsample_bytree': 0.8}
				 ------------------------
				 Best score

C:\Users\Lucas\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Lucas\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\Lucas\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[21:12:28] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
			 Time taken: 0 hours 0 minutes and 18.91 seconds.
				 Los mejores parametros son: 
				 {'subsample': 0.8, 'n_estimators': 50, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.01, 'gamma': 0.5, 'colsample_bytree': 0.8}
				 ------------------------
				 Best score: 
				 -100.19484912441465
				 ------------------------
	 drop other nans
	 Recast final
Comenzando operaciones
	 Nulls
		 Drop cols
		 Fill metros
			 Terreno
			 Apartamento
			 Metros cubiertos null
			 Metros totales null
			 Terreno comercial
	 Encoding
	 Cast
	 Features engineering
		 Separar fecha
		 Predict nulls
			 fill with xgboost. Feature:  {'garages'}
			 Time taken: 0 hours 0 minutes and 0.56 seconds.
			 fill with xgboost. Feature:  {'banos'}
			 Time taken: 0 hours 0 minutes and 0.57 seconds.
			 fill with xgboost. Feature:  {'habitaciones'}
			 Time taken: 0 hours 0 minutes and 1.

In [13]:
p = ParamsGrid()
# print(ParamsGrid.param_random_forest)

SyntaxError: invalid syntax (<ipython-input-13-6f404b5daa2a>, line 3)

In [15]:
param_random_forest = {
        'bootstrap': [True, False], #method for sampling data points (with or without replacement)
        #'criterion': 'mse',
        'max_depth': [None, 10, 20, 50, 75, 100], #max number of levels in each decision tree
        'max_features': ['auto', 'sqrt'],  # np.arange(10, 12, 15) #max number of features considered for splitting a node
        #'max_leaf_nodes': None,
        #'min_impurity_decrease': 0.0,
        #'min_impurity_split': None,
        'min_samples_leaf': np.arange(1, 2, 4), #min number of data points allowed in a leaf node
        'min_samples_split': np.arange(2, 5, 10), #min number of data points placed in a node before the node is split
        #'min_weight_fraction_leaf': 0.0,
        'n_estimators': np.arange(75, 100, 200),  # n_estimators = number of trees in the foreset
        #'n_jobs': 1,
        #'oob_score': False,
#         'random_state': 42,
#         'verbose': 0,
#         'warm_start': False
    }


In [11]:
print(data[0].shape)
print(data[1].shape)

(219442, 37)
(60000, 36)


In [16]:

model = Prediction(data, RandomForestRegressor(), param_random_forest, 'prediction')
model.train()
model.save()

Training...
{'bootstrap': False, 'max_depth': 100, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 75}
-621755.2063532729


In [18]:
model.submit()

In [19]:
answer = pd.DataFrame(list(zip(model.test_ids, model.predicted)), columns=['id', 'predicted'])
answer.to_csv('{}.csv'.format('result'), sep=',', index=False)


In [ ]:
# def nulls(df):
#     nulls = pd.DataFrame((df.isnull().sum().sort_values()/len(df)*100).round(2),columns=['porcentaje de NaN'])
#     if (nulls.sum() == 0).bool():
#         print("No hay nulls")
#     else:
#         nulls.drop(nulls.loc[nulls.loc[:,'porcentaje de NaN']<=0].index, inplace=True)
#         plt.figure(figsize=(12,8))
#         ax = nulls['porcentaje de NaN'].plot.barh()
#         ax.set_title('Porcentaje de valores nulos en cada columna',fontsize=20, y=1.02)
#         ax.set_xlabel('Porcentaje del total %', fontsize=16)
#         ax.set_ylabel('columnas', fontsize=16)
#         ax.grid(axis='x')

#         for y, x in enumerate(nulls['porcentaje de NaN']):
#             ax.text(x, y, s=str(x)+'%', color='black', fontweight='bold',va='center')

#         plt.show()

# df_test = pd.read_csv('data/test.csv')
# # nulls(df_test)
# nulls(dfs[0])
# nulls(dfs[1])

# dfs[1].info()
# out.df_xgb.garages_xgb.value_counts().sum()